# Word2Vec and Clustering

## 1. Word2Vec trainning
Considering that our corpus is relatively small, we chose 100 dimension vectors for word presentation, Both CBOW and Skip-gram models are trainned, negative sampling is used to accelerate. The trainning algorithm is deployed under the framework of gensim.

In [ ]:
Sentences = ReloadCorpus('D:/_Research/Project_Ecological_Development/Data_Processing/Sentences_TiAb_BiReplaced.txt')
logging.basicConfig(format='%(asctime)s : %(levelname)s :%(message)s', level=logging.INFO)
model = gensim.models.Word2Vec(Sentences, size = 100, min_count = 3, sg = 1, hs = 1)
model.save('D:/_Research/Project_Ecological_Development/Data_Processing/SkipGram_HierSoftmax.model')

model = gensim.models.Word2Vec(Sentences, size = 100, min_count = 3, sg = 0, hs = 0)
model.save('D:/_Research/Project_Ecological_Development/Data_Processing/CBOW_NegativeSampling.model')

# 2. Concept word group generation

First, delete all the stopwords in both corpuses using nltk.stopwords.words('english'), then lemmatize the corpus using nltk.WordNetLemaatizer (Attention, this step takes long time and delete stopwords first will accelerate this process.

In [3]:
#Function Declaration
def DelStopwords(data, UniBi): #For stopwords deletion in Corpus files
    stpwds = stopwords.words('english')    
    result = []
    if UniBi == 'Sent':
        for record in data:
            result.append([word for word in record if word not in stpwds])
    if UniBi == 'Uni':
        result = [word for word in data if word not in stpwds]
    return(result)

def Lemm(data, ps):#Data must be whole corpus, contains mutiple word Lists [[x,x,x],[x,x,x],...,[x,x,x]]Style
    result = []
    wnl = nltk.WordNetLemmatizer()
    if ps == 'v' or ps == 'n':
        for record in data:
            result.append([str(wnl.lemmatize(word, pos = ps)) for word in record]) #POS = 'v' make this lemmatizer process verbs and 'n' for nouns, Recommend both.
    if ps == 'all':
        for record in data:
            tempresult = []
            tempresult = [str(wnl.lemmatize(word, pos = 'n')) for word in record]
            result.append([str(wnl.lemmatize(word, pos = 'v')) for word in tempresult]) #Make both pos lemmatized
    return(result)

In [9]:
#Stopwords deletion
Title_Corpus = DelStopwords(Title_Corpus, 'Sent')
Abstract_Corpus = DelStopwords(Abstract_Corpus, 'Sent')

#Lemmalitization
Title_Corpus = Lemm(Title_Corpus, 'all')
Abstract_Corpus = Lemm(Abstract_Corpus, 'all')

Save this contemporary files into local disk.

In [ ]:
def OutputFiles(data, FileDirectory, UniBi): #For Signle word and Sentence Outputing
    Fileoutput = open(FileDirectory, 'w', encoding = u'utf-8')
    if UniBi == 'Uni':
        for record in data:
            print(record, file = Fileoutput)
    if UniBi == 'Bi':
        for record in data:
            print(record[0], record[1], file = Fileoutput)    
    if UniBi == 'Sent':
        for record  in data:
            print(Connect(record), file = Fileoutput)
    Fileoutput.close()
    
#Output
OutputFiles(Title_Corpus,'D:/_Research/Project_Ecological_Development/Data_Processing/Sentences_Title_Lemmstp.txt','Sent')
OutputFiles(Abstract_Corpus,'D:/_Research/Project_Ecological_Development/Data_Processing/Sentences_Abstract_Lemmstp.txt','Sent')

# 3. Bigram selection and corpus refining

Here in this step, we'll refining the corpus by selecting important bigrams using TF-IDF, and then replace them back into the original corpus, Top 1000 bigrams are selected after mannual check.

In [ ]:
#Here is the function declaration
def CombineBigrams(Bgrams):
    result = []
    for sentence in Bgrams:
        temp = []
        for word in sentence:
            temp.append((word[0] + '-' + word[1]))
        if temp != []:
            result.append(temp)
    return(result)

def ReturnTfidfRank(Sentences,Top):
    dictionary = gensim.corpora.Dictionary(Sentences)
    corpus = [dictionary.doc2bow(sentence) for sentence in Sentences]
    tfidf = gensim.models.TfidfModel(corpus)
    TokenList = []
    for document in corpus:
        if document != []:
            TokenList.append(str(dictionary.get(sorted(tfidf[document],key = lambda record: record[1], reverse = True)[0][0])))
    result = [word for word in Sort_Rebuild(FreqDist(TokenList))][0:Top]
    return(result)

def Sort_Rebuild(Freq):#Rebuild and Sort FreqDist files, make it easier to check
    Freq_Rebuild = [[word,Freq[word]] for word in Freq]
    Freq_Sorted = sorted(Freq_Rebuild, key = lambda record: record[1], reverse = True)
    return(Freq_Sorted)

In [ ]:
#Combine the title and abstract corpus
Lemm_Sentences = Title_Corpus + Abstract_Corpus

#Make Bigrams
Bgrams = []
for sentence in Lemm_Sentences:
    Bgrams.append(list(nltk.bigrams(sentence)))

#Chosing using TF-IDF
Bgrams = CombineBigrams(Bgrams)
Bgram_Top_List = ReturnTfidfRank(Bgrams, 1000)

Output these Top 1000 Bigrams into local disk.

In [ ]:
Bgram_Top_List = [word[0] for word in Bgram_Top]
OutputFiles(Bgram_Top_List,'D:/_Research/Project_Ecological_Development/Data_Processing/Top1000_Bgrams_TFIDF.txt','Uni')

Replace bigrams back into the corpus using loop, Here, we changed the loop style into 'temp in Bgram_Top_List', which made it nearly 10000 times faster than original compare by new hash table comparing.
Attention, the time consumed by this step is mainly controlled by the volume of Bgram_Top_List, at the complexity of O(n).

In [ ]:
#Bigram replacement
C_Lemm_Sentences = Lemm_Sentences
for i in range(0,len(Lemm_Sentences)):
    for j in range (0,len(Lemm_Sentences[i])-1):
        temp = Lemm_Sentences[i][j] + '-' + Lemm_Sentences[i][j+1]
        if (temp in Bgram_Top_List):
            C_Lemm_Sentences[i][j] = temp
            C_Lemm_Sentences[i][j+1] = ''

In [ ]:
OutputFiles(C_Lemm_Sentences,'D:/_Research/Project_Ecological_Development/Data_Processing/Sentences_TiAb_BiReplaced.txt','Sent')